In [15]:
import anndata as ad
from pycombat import Combat
import numpy as np
import pandas as pd
import os

In [16]:
correspondence = {}
correspondence['149Sm_CREB'] ='149Sm_pCREB'
correspondence['167Er_ERK'] ='167Er_pERK'
correspondence['164Dy_IkB'] ='164Dy_IkB'
correspondence['159Tb_MAPKAPK2'] ='159Tb_pMAPKAPK2'
correspondence['166Er_NFkB'] ='166Er_pNFkB'
correspondence['151Eu_p38'] ='151Eu_pp38'
correspondence['155Gd_S6'] ='155Gd_pS6'
correspondence['153Eu_STAT1'] ='153Eu_pSTAT1'
correspondence['154Sm_STAT3'] ='154Sm_pSTAT3'
correspondence['150Nd_STAT5'] ='150Nd_pSTAT5'
correspondence['168Er_pSTAT6'] ='168Er_pSTAT6'
correspondence['174Yb_HLADR'] ='174Yb_HLADR'
correspondence['169Tm_CD25'] ='169Tm_CD25'

In [17]:
def batch_correction_combat(correspondence,perio_path, surge_path,stim):
    perio = ad.read_h5ad(perio_path)
    surge = ad.read_h5ad(surge_path)
    # Load AnnData
    perio_unstim=perio[perio.obs['drug']=='Unstim']
    surge_unstim=surge[surge.obs['drug']=='Unstim']
    surge_stim=surge[surge.obs['drug']==stim]

    # Choose the features
    features=correspondence.values()
    features = list(features)
    surge_unstim = surge_unstim[:, features]
    surge_stim = surge_stim[:, features]

    # Rename the features in perio_unstim to match those in surge_unstim
    for marker in correspondence.keys():
        perio_unstim.var_names = perio_unstim.var_names.to_series().replace(correspondence).values
    perio_unstim = perio_unstim[:, features]
    assert perio_unstim.var_names.equals(surge_unstim.var_names)
    
    combined_unstim = ad.concat(
        {"perio": perio_unstim, "surge": surge_unstim},
        label="batch"
    )

    combat = Combat()
    X_corrected_unstim = combat.fit_transform(combined_unstim.X, combined_unstim.obs["batch"])

    surge_unstim_corrected = surge_unstim.copy()
    surge_unstim_corrected.X = X_corrected_unstim[combined_unstim.obs["batch"] == "surge"]

    # Create a dummy row for the perio batch
    dummy = surge_stim[:1].copy()  # copy one row from surge_stim
    dummy.X = np.zeros_like(dummy.X)  # zero values
    dummy.obs['batch'] = 'perio'

    # Concat dummy + real surge_stim
    combined_stim = ad.concat(
        {"surge": surge_stim, "perio": dummy},
        label="batch"
    )

    # Perform batch correction on the combined data
    X_surge_corrected = combat.transform(combined_stim.X,combined_stim.obs['batch'])  # Only transform, don't fit
    surge_stim_corrected = surge_stim.copy()
    surge_stim_corrected.X = X_surge_corrected[combined_stim.obs["batch"] == "surge"]

    surge_corrected=ad.concat(
        {"Unstim": surge_unstim_corrected, stim: surge_stim_corrected},
        label="stim"
    )

    surge_corrected = surge_corrected.copy()
    surge_corrected.strings_to_categoricals()

    for col in surge_corrected.obs.columns:
        if isinstance(surge_corrected.obs[col].dtype, pd.CategoricalDtype):
            surge_corrected.obs[col] = surge_corrected.obs[col].astype(str)
    surge_name= surge_path.split('/')[-1]
    surge_corrected.write('doms_corrected/'+surge_name)
    return

In [19]:
directory_perio='perio_data_sherlock2'
directory_surge='doms_concatenated'
listfile_perio=[f for f in os.listdir(directory_perio) if 'TNFa' not in f]
listfile_surge=os.listdir(directory_surge)
for filename_perio,filename_surge in zip(listfile_perio,listfile_surge):
    path_perio=os.path.join(directory_perio, filename_perio)
    path_surge=os.path.join(directory_surge, filename_surge)
    if 'LPS' in filename_surge:
        batch_correction_combat(correspondence, path_perio, path_surge,'LPS')
    else:
        batch_correction_combat(correspondence, path_perio, path_surge,'IFNa')
    print(f"Corrected {filename_perio} and {filename_surge}")

Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
... storing 'drug' as categorical
... storing 'drug' as categorical
... storing 'cell_type' as categorical
... storing 'patient' as categori

Corrected perio_data_sherlockIFNa_Classical Monocytes (CD14+CD16-)_train.h5ad and doms_concatenated_LPS_Intermediate Monocytes (CD14+CD16+).h5ad
Corrected perio_data_sherlockP. gingivalis_Intermediate Monocytes (CD14+CD16+)_train.h5ad and doms_concatenated_IFNa_B-Cells (CD19+CD3-).h5ad


Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
... storing 'drug' as categorical
... storing 'drug' as categorical
... storing 'cell_type' as categorical
... storing 'patient' as categorical
... storing 'stim' as categorical


Corrected perio_data_sherlockP. gingivalis_MDSCs (lin-CD11b-CD14+HLADRlo)_train.h5ad and doms_concatenated_LPS_mDCs (CD11c+HLADR+).h5ad


Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
... storing 'drug' as categorical
... storing 'drug' as categorical
... storing 'cell_type' as categorical
... storing 'patient' as categori

...

Corrected perio_data_sherlockIFNa_Granulocytes (CD45-CD66+)_train.h5ad and doms_concatenated_LPS_CD56loCD16+NK Cells.h5ad
Corrected perio_data_sherlockP. gingivalis_pDCs(CD123+HLADR+)_train.h5ad and doms_concatenated_IFNa_CD56+CD16- NK Cells.h5ad


Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
... storing 'drug' as categorical
... storing 'drug' as categorical
... storing 'cell_type' as categorical
... storing 'patient' as categori

Corrected perio_data_sherlockP. gingivalis_CD8 T-Cells_train.h5ad and doms_concatenated_IFNa_Intermediate Monocytes (CD14+CD16+).h5ad


Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
... storing 'drug' as categorical
... storing 'drug' as categorical
... storing 'cell_type' as categorical
... storing 'patient' as categori

Corrected perio_data_sherlockIFNa_CD4 T-Cells_train.h5ad and doms_concatenated_LPS_Non-classical Monocytes (CD14-CD16+).h5ad


In [4]:
batch_correction_combat(correspondence, 'perio_data_sherlock2/perio_data_sherlockP. gingivalis_mDCs (CD11c+HLADR+)_train.h5ad', 'surge_concatenated/surge_concatenated_LPS_mDCs (CD11c+HLADR+).h5ad','LPS')

Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
... storing 'drug' as categorical
... storing 'drug' as categorical
... storing 'cell_type' as categorical
... storing 'patient' as categori